<H1>PRICE PREDICTION OF PRODUCTS: HACKEREARTH HACKATHON</H1><BR>SUBMISSION-4:MULTIPLE MODELS <BR>GRID SEARCH TRIAL 2<BR>VISUALS NOT INCLUDED

IMPORTING FILES

In [1]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split

from sklearn import svm,tree,linear_model,naive_bayes,ensemble,gaussian_process,neighbors
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

from sklearn.model_selection import RepeatedKFold
from numpy import mean
from numpy import std
from sklearn.model_selection import GridSearchCV

READING DATA

In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6368 entries, 0 to 6367
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_id        6368 non-null   object 
 1   Stall_no          6363 non-null   float64
 2   instock_date      6368 non-null   object 
 3   Market_Category   6368 non-null   int64  
 4   Customer_name     6157 non-null   object 
 5   Loyalty_customer  6368 non-null   object 
 6   Product_Category  6368 non-null   object 
 7   Grade             6368 non-null   int64  
 8   Demand            6368 non-null   int64  
 9   Discount_avail    6330 non-null   float64
 10  charges_1         6170 non-null   float64
 11  charges_2 (%)     6163 non-null   float64
 12  Minimum_price     6330 non-null   float64
 13  Maximum_price     6025 non-null   float64
 14  Selling_Price     6327 non-null   float64
dtypes: float64(7), int64(3), object(5)
memory usage: 746.4+ KB
<class 'pandas.core.frame.Data

In [3]:
train_data.head()

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price
0,BRAE2NF6JA5GUEXG,37.0,2015-08-22 18:36:12.000,2,Lillyann,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700
1,TUNE8SFB6RJN2HSD,38.0,2016-03-27 21:19:13.000,24,Klynn,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256
2,BRAEAR7WZPQGPBZU,9.0,2015-08-18 19:25:22.000,447,Ridge,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362
3,WATDZ2ZQ8JPDHCTJ,50.0,2016-03-28 21:53:01.000,23,Abran,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824
4,JWSEBUKYQPMBZ3RK,7.0,2016-03-29 22:58:53.000,63,Dustyn,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935


CONVERTING DATE FROM OBJECT TO DATE-TIME
NEW FEATURE : INSTOCK YEAR 

In [4]:
for data in [train_data, test_data]:
    data['instock_date'] = data['instock_date'].str.split(" ", n=1,expand = True)[0]
    data['instock_date'] = pd.to_datetime(data['instock_date'])
    data['instock_year'] = data['instock_date'].dt.year
    data['instock_month'] = data['instock_date'].dt.month

In [5]:
for data in [train_data,test_data]:
    data['average_price'] = (data['Minimum_price']+data['Maximum_price'])/2

In [21]:
for data in [train_data,test_data]:
    data['average_diff'] = (data['Maximum_price']-data['Minimum_price'])/2

CREATING DUMMY VARIABLES FOR CATAGORIES

In [22]:
full_data = [train_data, test_data]
for data in full_data:
    data.drop(['Customer_name', 'Loyalty_customer','Stall_no', 'Market_Category', 'Demand','instock_date'], inplace = True, axis = 1)
    
Product_Category_encoded = pd.get_dummies(train_data['Product_Category'])
train_data = pd.concat([train_data,Product_Category_encoded], axis = 1)
Product_Category_encoded = pd.get_dummies(test_data['Product_Category'])
test_data = pd.concat([test_data,Product_Category_encoded], axis = 1)
train_data.head()

KeyError: "['Customer_name' 'Loyalty_customer' 'Stall_no' 'Market_Category' 'Demand'\n 'instock_date'] not found in axis"

TOTAL NUMBER OF NULL VALUES IN EACH COLUMN OF TRAIN AND TEST DATA
TOTAL NUMBER OF NULL VALUES IN EACH PRODUCT CATAGORY WITH RESPECT TO SELECTED COLUMNS

In [7]:
for data in full_data:
    for x in data.columns:
        print("{col} : {val}".format(col = x, val = data[x].isna().sum()))
    print('-'*30)

listProd = ['Child_care','Cosmetics', 'Educational','Fashion','Home_decor',	'Hospitality',	'Organic',	'Pet_care',	'Repair','Technology']
nullCols = ['Discount_avail','charges_1','charges_2 (%)','Minimum_price','Maximum_price']
for data in full_data:
    for y in nullCols:
        print(y)
        for x in listProd:
            print('{col}:  {val}'.format(col = x, val=(data['Product_Category'].loc[data[y].isna()] ==x).sum()))
        print('----------------------')
    print('=========================')




Product_id : 0
Product_Category : 0
Grade : 0
Discount_avail : 38
charges_1 : 198
charges_2 (%) : 205
Minimum_price : 38
Maximum_price : 343
Selling_Price : 41
instock_year : 0
instock_month : 0
average_price : 377
------------------------------
Product_id : 0
Product_Category : 0
Grade : 0
Discount_avail : 0
charges_1 : 36
charges_2 (%) : 5
Minimum_price : 14
Maximum_price : 0
instock_year : 0
instock_month : 0
average_price : 14
------------------------------
Discount_avail
Child_care:  6
Cosmetics:  2
Educational:  5
Fashion:  4
Home_decor:  5
Hospitality:  6
Organic:  2
Pet_care:  0
Repair:  6
Technology:  2
----------------------
charges_1
Child_care:  18
Cosmetics:  14
Educational:  17
Fashion:  25
Home_decor:  29
Hospitality:  22
Organic:  13
Pet_care:  15
Repair:  23
Technology:  22
----------------------
charges_2 (%)
Child_care:  20
Cosmetics:  26
Educational:  16
Fashion:  19
Home_decor:  22
Hospitality:  23
Organic:  23
Pet_care:  22
Repair:  23
Technology:  11
------------

DROPPING NULL ROWS

In [8]:
train_data.isna().sum()


Product_id            0
Product_Category      0
Grade                 0
Discount_avail       38
charges_1           198
charges_2 (%)       205
Minimum_price        38
Maximum_price       343
Selling_Price        41
instock_year          0
instock_month         0
average_price       377
Child_care            0
Cosmetics             0
Educational           0
Fashion               0
Home_decor            0
Hospitality           0
Organic               0
Pet_care              0
Repair                0
Technology            0
dtype: int64

In [23]:
values = {'Discount_avail': train_data['Discount_avail'].mean(), 'charges_1': train_data['charges_1'].mean(), 'charges_2 (%)':train_data['charges_2 (%)'].median() , 'Minimum_price': train_data['Minimum_price'].median(),'Maximum_price':train_data['Maximum_price'].median(),'average_price': train_data['average_price'].median(),'average_diff': train_data['average_diff'].median()}
train_data.fillna(value=values,inplace=True)

In [24]:
values = {'Discount_avail': test_data['Discount_avail'].mean(), 'charges_1': test_data['charges_1'].mean(), 'charges_2 (%)':test_data['charges_2 (%)'].median(),'Minimum_price': test_data['Minimum_price'].median(),'average_price': test_data['average_price'].median(),'average_diff': test_data['average_diff'].median()}
test_data.fillna(value=values,inplace=True)

In [25]:
train_data = train_data[~train_data['Selling_Price'].isna()]
train_data.isna().sum()

Product_id          0
Product_Category    0
Grade               0
Discount_avail      0
charges_1           0
charges_2 (%)       0
Minimum_price       0
Maximum_price       0
Selling_Price       0
instock_year        0
instock_month       0
average_price       0
Child_care          0
Cosmetics           0
Educational         0
Fashion             0
Home_decor          0
Hospitality         0
Organic             0
Pet_care            0
Repair              0
Technology          0
average_diff        0
dtype: int64

PREPARING TRAINING DATA AND TESTING DATA FOR MODEL SELECTION

In [54]:
def target_corrs(df):
    corrs = []
    for col in df.columns:
        print(col)
        if col != 'Selling_Price':
            corr = df['Selling_Price'].corr(df[col])
            corrs.append((col, corr))
    corrs = sorted(corrs, key = lambda x: abs(x[1]), reverse = True)
    
    return corrs

In [56]:
# train_data.drop(['Product_id','Product_Category'],axis =1,inplace=True)
target_corrs(train_data)

Grade
Discount_avail
charges_1
Minimum_price
Maximum_price
Selling_Price
instock_year
average_diff


[('Maximum_price', 0.7554972726139902),
 ('Minimum_price', 0.7037353624270787),
 ('Discount_avail', -0.5185286576211869),
 ('average_diff', 0.4528579320410935),
 ('Grade', 0.08530436537150561),
 ('instock_year', 0.053254657666356835),
 ('charges_1', 0.05254494479724394)]

DROPPING UNRELATED COLUMNS 

In [ ]:
train_data.drop(['instock_month', 'average_price',
       'Child_care', 'Cosmetics', 'Educational', 'Fashion', 'Home_decor',
       'Hospitality', 'Organic', 'Pet_care', 'Repair', 'Technology','charges_2 (%)'],axis=1,inplace = True)'Grade', 'Discount_avail',
       'charges_1', 'charges_2 (%)', 'Minimum_price', 'Maximum_price',
       'instock_year',

In [36]:
train_data = train_data[train_data['Selling_Price']>0]
X = train_data
y = train_data['Selling_Price']
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [57]:
train_data.drop(['Discount_avail'],inplace = True, axis = 1)

RMLSE SCORE FUNCTION

In [37]:
from sklearn.metrics import mean_squared_log_error
def RMSLE(y_true:np.ndarray, y_pred:np.ndarray) -> np.float64:
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

TESTING REGRESSION MODELS FOR BEST MODEL SELECTION

In [38]:
MLA = [
    ensemble.ExtraTreesRegressor(),
    ensemble.RandomForestRegressor(),
    XGBRegressor(),
    LGBMRegressor()    
    ]

CHECKING MINIMUM RMLSE OUT OF ALL MODEL PREDICTIONS

In [39]:
from sklearn.model_selection import cross_val_score
for model in MLA:
    scores = cross_val_score(model, X, y, cv=5, scoring = 'max_error')
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: -1223.44 (+/- 1150.37)
Accuracy: -992.28 (+/- 1566.97)
Accuracy: -629.43 (+/- 902.31)
Accuracy: -2602.30 (+/- 2479.95)


In [62]:
parameters = [{
        'n_estimators' : [50,100,130,150,200],
         'max_features': [None],
         'max_depth': [3,4,5,6,7],
         'min_samples_split' : [2,4,6,8],
         'max_features' : ('auto','sqrt','log2',None),
         'n_jobs' : [-1]
    }]
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
best_search = GridSearchCV(estimator = ensemble.ExtraTreesRegressor(), param_grid=parameters, scoring='max_error',cv = cv)
best_search.fit(X_train,y_train)
print(best_search.best_score_)
print(best_search.best_params_)

-1029.707757784308
{'max_depth': 7, 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 200, 'n_jobs': -1}


In [58]:
model = ensemble.ExtraTreesRegressor(n_estimators = 150,max_features=None,max_depth= None,min_samples_split= 2,n_jobs = -1)
best_search1 = model.fit(X_train,y_train)
pred = best_search1.predict(X_test)
data = {'y-test':y_test, 'predictions':pred}
df = pd.DataFrame(data=data, columns=['y-test','predictions'])
df = df[df['predictions']>0]
score = RMSLE(df['y-test'], df['predictions'])
score 

0.03272041074250021

In [42]:
parameters = [{
        'learning_rate': [.03, 0.05, .07],
        'max_depth' : [3,5,7,9],
        'gamma' : [0,0.01,0.1],
        'colsample_bytree': [0.5,0.7,1.0],
        'objective': ['reg:squarederror'],
        'eval_metric' : ['mae'],

    }]
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
best_search = GridSearchCV(estimator = XGBRegressor(), param_grid=parameters, scoring='max_error',cv = cv)
best_search.fit(X_train,y_train)
print(best_search.best_score_)
print(best_search.best_params_)

-481.8678166666663
{'colsample_bytree': 1.0, 'eval_metric': 'mae', 'gamma': 0, 'learning_rate': 0.07, 'max_depth': 5, 'objective': 'reg:squarederror'}


In [59]:
model = XGBRegressor(colsample_bytree= 1.0, eval_metric= 'mae', gamma= 0, learning_rate= 0.07, max_depth= 5, objective='reg:squarederror')
best_search = model.fit(X_train,y_train)
pred = best_search.predict(X_test)
data = {'y-test':y_test, 'predictions':pred}
df = pd.DataFrame(data=data, columns=['y-test','predictions'])
df = df[df['predictions']>0]
score = RMSLE(df['y-test'], df['predictions'])
score 

0.003790336953726824

In [61]:
features = ['Grade', 'Discount_avail', 'charges_1', 'Minimum_price', 'Maximum_price', 'Selling_Price', 'instock_year', 'average_diff']
x = test_data[features]
predictions = best_search.predict(x)
test_data['Selling_Price'] = predictions
data = {'Product_id':test_data['Product_id'], 'Selling_Price':test_data['Selling_Price']}
submissions_df = pd.DataFrame(data= data, columns = ['Product_id', 'Selling_Price'])
submissions_df.to_csv(path_or_buf='/Users/username/Desktop/source/submission.csv')